Imports

In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import time

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

## Baseline LSTM + room to change network architecture

In [3]:
def handle_wind_dir(data):
    '''
    Different ways to handle the string format:
     1. Drop it
     2. LabelEncode it
     3. One-hot encode it
     
    The reason I need to handle it is because Keras' pad_sequences function takes int() of
    all the columns.
    '''
    # I'll drop it first
    data = data.drop('wnd_dir', axis=1, inplace=False)
    return data

In [4]:
import warnings; warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('pollution.csv', header=0, index_col=0)

In [6]:
# Optionally drop wind dir
df = handle_wind_dir(df)

In [7]:
train, test = train_test_split(df, test_size=.998, random_state=789)

In [8]:
len(train)

87

In [9]:
len(test)

43713

In [10]:
def prepare_sequences(df):
    ''' 
    Let df = train and prepare sequences.
    '''
    input_cols = list(df.columns)
    df['single_input_vector'] = df[input_cols].apply(tuple, axis=1).apply(list)\
        .apply(lambda x: [list(x)])
    return df.reset_index(drop=True)

In [11]:
train = prepare_sequences(train)

In [12]:
train.head()

,pollution,dew,temp,press,wnd_spd,snow,rain,single_input_vector
0,16.0,10,14.0,1020.0,18.32,0,0,"[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0]]"
1,12.0,-11,2.0,1025.0,109.07,0,0,"[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0]]"
2,0.0,17,21.0,1012.0,48.27,0,0,"[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0]]"
3,68.0,17,19.0,1006.0,3.58,0,0,"[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0]]"
4,0.0,-19,5.0,1017.0,32.62,0,0,"[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0]]"


In [82]:
def func(x, maxlen=None):
    '''
    Pad sequences with lists of 0s. Functional lambda programming.
    '''
    zeros_to_add = maxlen - len(x)
    prepended = [np.zeros(7).tolist()]
    y = prepended*zeros_to_add + x
    return y

def balanced_sliding_windows(df, stride=3):
    '''
    Create sliding windows of size [n - 3, n + 3]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i-stride:i+stride, 0].sum()
        elif i < stride:
            sequences = df.iloc[i:i+stride, 0].sum()
        else:
            sequences = df.iloc[i-stride:i,].sum()
        seqs.append(sequences)
    return pd.Series(seqs)
        
def past_windows(df, stride=3):
    '''
    Create windows of size [n - 3: n]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i-stride:i, -1].sum()
        elif i < stride:
            sequences = df.iloc[0:i, -1].sum()
        else:
            sequences = df.iloc[i-stride:i, -1].sum()
        if sequences == 0:
            # Pad empty array with arrays of 0s
            sequences = func([], stride)
        if len(sequences) < stride:
            # Pad with arrays of 0s
            sequences = func(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

def future_windows(df, stride=3):
    '''
    Create windows of size [n : n+3]. Let df=train.
    '''
    seqs = []
    for i, value in df.iterrows():
        if i >= stride and i < len(df) - stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        elif i < stride:
            sequences = df.iloc[i:i+stride, -1].sum()
        else:
            sequences = df.iloc[i:, -1].sum()
        if type(sequences) == 'int' and sequences == 0:
            sequences = func([], stride)
        if len(sequences) < stride:
            # Pad with arrays of 0s
            sequences = func(sequences, stride)
        seqs.append(sequences)
    return pd.Series(seqs)

In [83]:
input_vec = train['single_input_vector']
input_vec = pd.DataFrame(input_vec)

In [84]:
# Good so far.

In [85]:
balanced_sliding_windows(input_vec).head()

0    [[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...
1    [[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...
2    [[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [...
3    [[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...
4    [[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...
dtype: object

In [86]:
past_windows(input_vec).head()

0    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....
1    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....
2    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [16.0, 1...
3    [[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...
4    [[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...
dtype: object

In [87]:
future_windows(input_vec).head()

0    [[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...
1    [[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...
2    [[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [...
3    [[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0], [...
4    [[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0], [...
dtype: object

In [88]:
# Good so far

In [89]:
train['past_sequences'] = past_windows(input_vec)
train['future_sequences'] = future_windows(input_vec)

In [90]:
train.head()

,pollution,dew,temp,press,wnd_spd,snow,rain,single_input_vector,past_sequences,future_sequences,output_vector
0,16.0,10,14.0,1020.0,18.32,0,0,"[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ..."
1,12.0,-11,2.0,1025.0,109.07,0,0,"[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],..."
2,0.0,17,21.0,1012.0,48.27,0,0,"[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [16.0, 1...","[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [...","[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [..."
3,68.0,17,19.0,1006.0,3.58,0,0,"[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0]]","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...","[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0], [...","[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0], [..."
4,0.0,-19,5.0,1017.0,32.62,0,0,"[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0]]","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...","[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0], [...","[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0], [..."


In [91]:
# predicting G for now, just a test example
# If your output is multi-dimensional, you need to capture those 
# dimensions in one object
# If your output is a single dimension, this step may be unnecessary
def set_output_featureset(df):
    '''
    Let df=train and prepare output vector
    '''
    df['output_vector'] = df['future_sequences']
    return df

In [92]:
train = set_output_featureset(train)

In [93]:
train.head()

,pollution,dew,temp,press,wnd_spd,snow,rain,single_input_vector,past_sequences,future_sequences,output_vector
0,16.0,10,14.0,1020.0,18.32,0,0,"[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ..."
1,12.0,-11,2.0,1025.0,109.07,0,0,"[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],..."
2,0.0,17,21.0,1012.0,48.27,0,0,"[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0]]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [16.0, 1...","[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [...","[[0.0, 17.0, 21.0, 1012.0, 48.27, 0.0, 0.0], [..."
3,68.0,17,19.0,1006.0,3.58,0,0,"[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0]]","[[16.0, 10.0, 14.0, 1020.0, 18.32, 0.0, 0.0], ...","[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0], [...","[[68.0, 17.0, 19.0, 1006.0, 3.58, 0.0, 0.0], [..."
4,0.0,-19,5.0,1017.0,32.62,0,0,"[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0]]","[[12.0, -11.0, 2.0, 1025.0, 109.07, 0.0, 0.0],...","[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0], [...","[[0.0, -19.0, 5.0, 1017.0, 32.62, 0.0, 0.0], [..."


In [94]:
# Extract your training data
X_train_init = np.asarray(train.past_sequences)
stride_length = 3
# Training data for LSTM should be in the form of a 3D tuple:
#   (# of samples, timesteps, input_dim)
# Note that the input data that comes out of the dataframe 
# will not make a 3D array. It makes an array of arrays, 
# which is not the same thing.
# So far X_train_init is an array of arrays.

# Convert to 3D vector usng hstack and reshape
# horizontal stack = hstack, essentially removes the outer array 
#  encapsulation.

# reshape into (# of records, total_timesteps, input_dim)
# The reshape essentially reshaped the inner list into an 11 by 6 matrix,
#  or a max_sequence_length (rows) by input_dim (col) matrix
X_train = np.hstack(X_train_init).reshape(len(train), 
                                          stride_length,
                                         len(df.columns))
y_train_init = np.asarray(train.output_vector)
y_train = np.hstack(y_train_init).reshape(len(train), 
                                          stride_length,
                                          len(df.columns))

In [95]:
len(train), len(df.columns)

(87, 7)

In [97]:
# Debugging
#train[train.output_vector.apply(lambda x: True if len(x) != 3 else False)]

In [98]:
print(X_train.shape)
print(y_train.shape)

(87, 3, 7)
(87, 3, 7)


In [99]:
# Get your input dimensions
# Input length is the length for one input sequence 
#  (i.e. the number of rows for your sample, which is
#     the max_sequence_length by construction)
input_length = X_train.shape[1]
# Input dim is the number of dimensions in one input vector 
#  (i.e. number of input columns)
input_dim = X_train.shape[2]
output_dim = len(y_train[0])

In [102]:
output_dim

3

In [106]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense

# Initialize the model
model = Sequential()

# arbitrarily picked the output dim to be 100
model.add(LSTM(100, input_shape=(input_length, input_dim), return_sequences=True))
# The max output value is > 1 so used relu as final activation
model.add(Dense(output_dim, activation='relu'))

model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])


In [107]:
# Set batch_size to 30 to show that it doesn't have to be a factor 
# or multiple of your sample size
history = model.fit(X_train, y_train,
                   batch_size = 30, epochs=50,
                   verbose = 1)

ValueError: Error when checking target: expected dense_3 to have shape (None, 3, 3) but got array with shape (87, 3, 7)

In [328]:
model.predict(X_train)

array([[  6.35896730e+00,   7.97015727e-02,   0.00000000e+00,
          8.13524055e+00,   5.88157415e+00,   0.00000000e+00,
          0.00000000e+00],
       [  6.33025789e+00,   2.58132249e-01,   0.00000000e+00,
          8.14544964e+00,   5.60397673e+00,   0.00000000e+00,
          0.00000000e+00],
       [  6.34985209e+00,   2.62183487e-01,   0.00000000e+00,
          8.16323662e+00,   5.60958290e+00,   0.00000000e+00,
          0.00000000e+00],
       [  6.35255146e+00,   2.62741566e-01,   0.00000000e+00,
          8.16568661e+00,   5.61035538e+00,   0.00000000e+00,
          0.00000000e+00],
       [  6.35291815e+00,   2.62817293e-01,   0.00000000e+00,
          8.16601944e+00,   5.61046028e+00,   0.00000000e+00,
          0.00000000e+00],
       [  6.22186041e+00,   3.04498151e-03,   0.00000000e+00,
          8.03037357e+00,   5.32202387e+00,   0.00000000e+00,
          8.38674456e-02],
       [  6.22186708e+00,   3.04635242e-03,   0.00000000e+00,
          8.03038025e+00,   5.32

In [329]:
y_train

array([[  1.60000000e+01,   1.00000000e+01,   1.40000000e+01,
          1.02000000e+03,   1.83200000e+01,   0.00000000e+00,
          0.00000000e+00],
       [  1.20000000e+01,  -1.10000000e+01,   2.00000000e+00,
          1.02500000e+03,   1.09070000e+02,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   1.70000000e+01,   2.10000000e+01,
          1.01200000e+03,   4.82700000e+01,   0.00000000e+00,
          0.00000000e+00],
       [  6.80000000e+01,   1.70000000e+01,   1.90000000e+01,
          1.00600000e+03,   3.58000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,  -1.90000000e+01,   5.00000000e+00,
          1.01700000e+03,   3.26200000e+01,   0.00000000e+00,
          0.00000000e+00],
       [  3.90000000e+02,  -8.00000000e+00,   0.00000000e+00,
          1.03100000e+03,   8.90000000e-01,   0.00000000e+00,
          0.00000000e+00],
       [  6.00000000e+00,   1.00000000e+00,   2.00000000e+00,
          1.02800000e+03,   8.90